In [1]:
import seldon.pipeline.sklearn_transform as ssk
import seldon.pipeline.auto_transforms as auto
import pandas as pd


df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",
                 header=None,names=["target","Alcohol","Malic acid","Ash","Alcalinity of ash","Magnesium","Total phenols","Flavanoids",
                         "Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue","OD280/OD315 of diluted wines",
                         "Proline"])


In [2]:
df.head()

,target,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


Create an auto transform to scale numeric columns automatically.

In [5]:
df["target"] = df["target"] - 1
t_auto = auto.Auto_transform(exclude=["target"])
df2 = t_auto.fit_transform(df)
df2.head()

fitting scaler for col  Alcohol
fitting scaler for col  Malic acid
fitting scaler for col  Ash
fitting scaler for col  Alcalinity of ash
fitting scaler for col  Magnesium
fitting scaler for col  Total phenols
fitting scaler for col  Flavanoids
fitting scaler for col  Nonflavanoid phenols
fitting scaler for col  Proanthocyanins
fitting scaler for col  Color intensity
fitting scaler for col  Hue
fitting scaler for col  OD280/OD315 of diluted wines
fitting scaler for col  Proline
num columns to drop  0
num scalers 13
num categorical  0
num dates 0
num date diffs 0
num bool 0
scaling col  OD280/OD315 of diluted wines 1 / 13
scaling col  Hue 2 / 13
scaling col  Nonflavanoid phenols 3 / 13
scaling col  Malic acid 4 / 13
scaling col  Alcohol 5 / 13
scaling col  Total phenols 6 / 13
scaling col  Color intensity 7 / 13
scaling col  Proanthocyanins 8 / 13
scaling col  Magnesium 9 / 13
scaling col  Ash 10 / 13
scaling col  Proline 11 / 13
scaling col  Alcalinity of ash 12 / 13
scaling col  Flavan

,target,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,0,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


Create an XGBoost classifier and run 5-fold cross validation on the data.

In [8]:
from seldon import xgb
import seldon.pipeline.cross_validation as cf
xgb = xgb.XGBoostClassifier(target="target",learning_rate=0.1,silent=0,objective='binary:logistic')
cv = cf.Seldon_KFold(xgb,5)
cv.fit(df2)

Estimator init called
finished include/exclude 1447580180.24
{'reg_alpha': 0, 'colsample_bytree': 1, 'silent': 0, 'colsample_bylevel': 1, 'scale_pos_weight': 1, 'learning_rate': 0.1, 'missing': None, 'max_delta_step': 0, 'nthread': -1, 'base_score': 0.5, 'n_estimators': 100, 'subsample': 1, 'reg_lambda': 1, 'seed': 0, 'min_child_weight': 1, 'objective': 'binary:logistic', 'max_depth': 3, 'gamma': 0}
finished include/exclude 1447580180.44
finished include/exclude 1447580180.46
{'reg_alpha': 0, 'colsample_bytree': 1, 'silent': 0, 'colsample_bylevel': 1, 'scale_pos_weight': 1, 'learning_rate': 0.1, 'missing': None, 'max_delta_step': 0, 'nthread': -1, 'base_score': 0.5, 'n_estimators': 100, 'subsample': 1, 'reg_lambda': 1, 'seed': 0, 'min_child_weight': 1, 'objective': 'binary:logistic', 'max_depth': 3, 'gamma': 0}
finished include/exclude 1447580180.65
finished include/exclude 1447580180.66
{'reg_alpha': 0, 'colsample_bytree': 1, 'silent': 0, 'colsample_bylevel': 1, 'scale_pos_weight': 1,

Seldon_KFold(clf=XGBoostClassifier(base_score=None,
         clf=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_a...torizer=DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True)),
       k=5)